![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [ ]:
qb = QuantBook()

dividends = qb.add_data(EODHDUpcomingDividends, "dividends").symbol

be_tested = qb.history(dividends, datetime(2015, 1, 1), datetime(2025, 2, 20), Resolution.DAILY)
cleaned_be_tested = be_tested.reset_index()[["symbol", "dividenddate", "dividend"]].groupby(["symbol", "dividenddate"]).last()
cleaned_be_tested

In [17]:
symbols = list(set(be_tested.index.get_level_values(0)))
benchmark = qb.history(Dividend, symbols, datetime(2015, 1, 1), datetime(2025, 2, 20), Resolution.DAILY)
cleaned_benchmark = benchmark.reset_index()[["symbol", "time", "distribution"]]
cleaned_benchmark.columns = ["symbol", "dividenddate", "distribution"]
cleaned_benchmark = cleaned_benchmark.groupby(["symbol", "dividenddate"]).last()
cleaned_benchmark

In [18]:
def create_confusion_maxtrix(cleaned_be_tested, cleaned_benchmark, N, date_since=None):
    if date_since:
        cleaned_be_tested = cleaned_be_tested[cleaned_be_tested.index.get_level_values(1) >= date_since]
        cleaned_benchmark = cleaned_benchmark[cleaned_benchmark.index.get_level_values(1) >= date_since]

    index_to_be_tested = list(cleaned_benchmark.index)
    k = 0
    while k < N:
        index_to_be_tested += list(pd.MultiIndex.from_tuples([(s, i+timedelta(k)) for (s, i) in cleaned_benchmark.index])) + list(pd.MultiIndex.from_tuples([(s, i-timedelta(k)) for (s, i) in cleaned_benchmark.index]))
        k += 1

    tp = cleaned_be_tested[cleaned_be_tested.index.isin(index_to_be_tested)]
    fn = cleaned_be_tested[~cleaned_be_tested.index.isin(index_to_be_tested)]

    index_to_be_tested = list(cleaned_be_tested.index)
    k = 0
    while k < N:
        index_to_be_tested += list(pd.MultiIndex.from_tuples([(s, i+timedelta(k)) for (s, i) in cleaned_be_tested.index])) + list(pd.MultiIndex.from_tuples([(s, i-timedelta(k)) for (s, i) in cleaned_benchmark.index]))
        k += 1

    fp = cleaned_benchmark[~cleaned_benchmark.index.isin(index_to_be_tested)]

    # Create confusion matrix
    # True Negatives (TN) is set to 0
    confusion_matrix = np.array([[tp.shape[0], fn.shape[0]], [fp.shape[0], 0]])
    return confusion_matrix

In [ ]:
cm = create_confusion_maxtrix(cleaned_be_tested, cleaned_benchmark, 1)
print(f"Accuracy: {cm[0][0]/np.sum(cm)}, Missed Rate: {cm[0][1]/np.sum(cm)}")

In [21]:
cm = create_confusion_maxtrix(cleaned_be_tested, cleaned_benchmark, 3)
print(f"±3 days Accuracy: {cm[0][0]/np.sum(cm)}, Missed Rate: {cm[0][1]/np.sum(cm)}")

In [22]:
cm = create_confusion_maxtrix(cleaned_be_tested, cleaned_benchmark, 7)
print(f"±7 days Accuracy: {cm[0][0]/np.sum(cm)}, Missed Rate: {cm[0][1]/np.sum(cm)}")